In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chat import process_input,initialize
from tarot import interpret_cards,general_reading
import requests
import threading

app = Flask(__name__)
CORS(app)

# 메인 챗봇 라우트
@app.route("",method=["POST"])
def chat():
    data = request.json
    user_nick = data.get("user_nick")
    user_input = data.get("input")

    if user_input:
        try:
            # chat 초기화
            initialize()
            
            response = process_input(user_nick,user_input)
            
            if response["reason"]:
                dic_data = {
                    "user_nick" : user_nick,
                    "user_input" : user_input
                }
                send_dic(dic_data)
            return response   
           
        except Exception as e:
            return jsonify({"error" : str(e)}), 500
        
    else:
        return jsonify({"error" : "사용자의 입력이 없습니다."}), 400
    
# 타로카드 라우트
@app.route("",method=["POST"])
def tarot_card():
    data = request.json
    cards = data.get("cards")
    reason = data.get("reason")
    user_input = data.get("user_input")
    user_select = data.get("select_cate")

    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    try:
        if reason:
            response = interpret_cards(user_input,cards)
        else:
            response = general_reading(user_select,cards)
        return response
    
    except Exception as e:
        return jsonify({"error":str(e)}),500

# reason이 있는경우에만 동작, 사용자 대답을 위해 비동기처리    
def send_dic(data):
    def user_data():
        sentiment_url = ""
        try:
            response = request.post(sentiment_url,json=data)
            response.raise_for_status()
        except requests.RequestException as e:
            e
    thread = threading.Thread(target=user_data)
    thread.start()


    

if __name__ == "__main__":
    app.run(port=) #Flask 서버 포트 설정